In [1]:
### cria uma streaming Kafka para ser executado a cada 30 segundos

SPARK_VERSION = '3.1.2'
SCALA_VERSION = '2.12'

import findspark
findspark.add_packages(['org.apache.spark:spark-sql-kafka-0-10_' + SCALA_VERSION + ':' + SPARK_VERSION ])

findspark.init()
 
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
sc = SparkContext(appName="PythonSparkStreamingKafka")
ssc = StreamingContext(sc, 30)
topicPartion = ('saulo-teste',0)
fromOffset = {topicPartion: 0}


:: loading settings :: url = jar:file:/home/saulos_pires/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/saulos_pires/.ivy2/cache
The jars for the packages stored in: /home/saulos_pires/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-26dacd19-17a6-42d5-95f3-065af201a484;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 402ms :: artifacts dl 17ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 

In [2]:

#### cria uma instancia Delta table 
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("AppSaulo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", 100000)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
#### inicializa o streaming e seleciona somente os campos key e value 


def ProcessaLinha(df_grava):
    #df_2 = df_grava.write.format("csv").mode("overwrite").save("tmp/file.csv")
    print(df_grava)



df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "saulo-teste") \
  .load()
  
campos = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [5]:
def ProcessaLinha(df_grava):
    pass


grava = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .foreach(ProcessaLinha) \
    .start()

22/05/29 14:56:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3be24bdd-9f36-4af9-ab1a-47d9fc280cac. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/29 14:56:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/29 14:56:58 ERROR MicroBatchExecution: Query [id = f09619c6-d265-42f6-857e-c2024419fa14, runId = 4031eaaa-7b5c-45e4-a8ac-0d17630e8f7f] terminated with error
org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1454)
	at org.apache.spark.sql.connector.c